In [ ]:
import os
import cv2
import pandas as pd
from pathlib import Path

In [ ]:
# Directorios
root_dir = "all"  # Carpeta con las imágenes originales
out_dir = "sorted_characters"  # Carpeta de salida para los recortes sin relleno

# Función para vaciar una carpeta sin eliminarla
def clear_folder(folder):
    if os.path.exists(folder):
        for file in os.listdir(folder):
            file_path = os.path.join(folder, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)

# Crear `sorted_characters` si no existe y vaciarlo si ya existe
os.makedirs(out_dir, exist_ok=True)
clear_folder(out_dir)

# Recorrer todas las carpetas dentro de `all`
for subfolder in os.listdir(root_dir):
    subfolder_path = os.path.join(root_dir, subfolder)

    if not os.path.isdir(subfolder_path):
        continue  # Saltar si no es una carpeta

    images_path = os.path.join(subfolder_path, "images")
    csv_path = None

    # Buscar el archivo CSV en la carpeta
    for file in os.listdir(subfolder_path):
        if file.endswith(".csv"):
            csv_path = os.path.join(subfolder_path, file)
            break

    if not csv_path or not os.path.exists(images_path):
        continue  # Si no hay CSV o carpeta de imágenes, saltar

    # Cargar las coordenadas desde el CSV
    df = pd.read_csv(csv_path)

    # Recorrer todas las imágenes en la carpeta "images"
    for image_file in os.listdir(images_path):
        image_path = os.path.join(images_path, image_file)

        if not image_file.lower().endswith(('png', 'jpg', 'jpeg')):
            continue  # Ignorar archivos que no sean imágenes

        # Cargar la imagen en escala de grises
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image_id = Path(image_file).stem
        char_data = df[df['Image'] == image_id]

        for _, row in char_data.iterrows():
            unicode_char = str(row['Unicode'])  # Asegurar que es una cadena
            x_min, y_min = row['X'], row['Y']
            width, height = row['Width'], row['Height']
            x_max, y_max = x_min + width, y_min + height

            # Recortar la imagen sin modificarla
            char_crop = image[y_min:y_max, x_min:x_max]

            # Crear la carpeta del Unicode si no existe
            unicode_dir = os.path.join(out_dir, unicode_char)
            os.makedirs(unicode_dir, exist_ok=True)

            # Guardar la imagen recortada en sorted_characters
            output_path = os.path.join(unicode_dir, f"{image_id}_{row.name}.png")
            cv2.imwrite(output_path, char_crop)

print("Recortes generados en 'sorted_characters'.")

In [ ]:
import os
import shutil

# Rutas de las carpetas de origen
sorted_characters_path = "sorted_characters"
kkanji2_path = "kkanji2"
output_path = "filtered_characters"

# Crear la carpeta de salida si no existe
os.makedirs(output_path, exist_ok=True)

# Obtener la lista de carpetas en cada directorio
sorted_folders = set(os.listdir(sorted_characters_path))
kkanji2_folders = set(os.listdir(kkanji2_path))

# Encontrar las carpetas en sorted_characters que existen en kkanji2
matching_folders = sorted_folders.intersection(kkanji2_folders)

# Copiar las carpetas coincidentes a la nueva ubicación
for folder in matching_folders:
    src_folder = os.path.join(sorted_characters_path, folder)
    dest_folder = os.path.join(output_path, folder)
    shutil.copytree(src_folder, dest_folder, dirs_exist_ok=True)

print(f"Se han copiado {len(matching_folders)} carpetas a '{output_path}'")


In [ ]:
import os
import cv2
import numpy as np
import shutil

# Directorios
filtered_dir = "filtered_characters"  # Carpeta con las imágenes filtradas
output_dir = "Datos_Kanjis"  # Carpeta de salida para imágenes procesadas

# Tamaño final de las imágenes
final_size = 64

# Función para vaciar una carpeta sin eliminarla
def clear_folder(folder):
    if os.path.exists(folder):
        for file in os.listdir(folder):
            file_path = os.path.join(folder, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)

# Crear `Datos_Kanjis` si no existe y vaciarlo si ya existe
os.makedirs(output_dir, exist_ok=True)
clear_folder(output_dir)

# Función para generar variaciones con zoom out, rotación y traslación
def transform_image(image, border_color, zoom_range=(0.4, 0.8), rotation_range=(-15, 15)):
    scale_factor = np.random.uniform(*zoom_range)
    new_width = max(1, int(image.shape[1] * scale_factor))
    new_height = max(1, int(image.shape[0] * scale_factor))
    scaled_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)

    transformed_image = np.full((final_size, final_size), border_color, dtype=np.uint8)
    x_space = final_size - new_width
    y_space = final_size - new_height
    x_offset = np.random.randint(0, x_space + 1) if x_space > 0 else 0
    y_offset = np.random.randint(0, y_space + 1) if y_space > 0 else 0

    transformed_image[y_offset:y_offset+new_height, x_offset:x_offset+new_width] = scaled_image

    angle = np.random.uniform(*rotation_range)
    rotation_matrix = cv2.getRotationMatrix2D((final_size // 2, final_size // 2), angle, 1)
    rotated_image = cv2.warpAffine(transformed_image, rotation_matrix, (final_size, final_size), borderValue=border_color)

    return rotated_image

# Procesar cada carpeta en `filtered_characters`
for unicode_char in os.listdir(filtered_dir):
    unicode_dir = os.path.join(filtered_dir, unicode_char)
    if not os.path.isdir(unicode_dir):
        continue

    unicode_output_dir = os.path.join(output_dir, unicode_char)
    os.makedirs(unicode_output_dir, exist_ok=True)
    # Procesar cada imagen en cada carpeta
    for image_file in os.listdir(unicode_dir):
        image_path = os.path.join(unicode_dir, image_file)
        if not image_file.lower().endswith(('png', 'jpg', 'jpeg')):
            continue

        char_crop = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Calcular el color del fondo basado en los bordes
        left_border = char_crop[:, 0]
        right_border = char_crop[:, -1]
        top_border = char_crop[0, :]
        bottom_border = char_crop[-1, :]
        all_borders = np.concatenate((left_border, right_border, top_border, bottom_border))
        border_color = np.percentile(all_borders, 90)

        # Redimensionar la imagen original sin deformarla
        h, w = char_crop.shape
        scale_factor = min(final_size / h, final_size / w)
        new_width = int(w * scale_factor)
        new_height = int(h * scale_factor)
        resized_image = cv2.resize(char_crop, (new_width, new_height), interpolation=cv2.INTER_AREA)

        # Crear imagen de 64x64 con fondo adaptado
        padded_image = np.full((final_size, final_size), border_color, dtype=np.uint8)
        x_offset = (final_size - new_width) // 2
        y_offset = (final_size - new_height) // 2
        padded_image[y_offset:y_offset+new_height, x_offset:x_offset+new_width] = resized_image

        # Generar dos imágenes transformadas
        transformed_image1 = transform_image(padded_image, border_color)
        transformed_image2 = transform_image(padded_image, border_color)

        # Guardar imágenes en la carpeta correspondiente
        base_name = os.path.splitext(image_file)[0]
        cv2.imwrite(os.path.join(unicode_output_dir, f"{base_name}_original.png"), cv2.bitwise_not(padded_image))
        cv2.imwrite(os.path.join(unicode_output_dir, f"{base_name}_var1.png"), cv2.bitwise_not(transformed_image1))
        cv2.imwrite(os.path.join(unicode_output_dir, f"{base_name}_var2.png"), cv2.bitwise_not(transformed_image2))

print("Imágenes generadas y guardadas en 'Datos_Kanjis'.")